## ThinkDSP

This notebook contains code examples from Chapter 2: Harmonics

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

%matplotlib inline

import thinkdsp
import thinkplot

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

## Waveforms and harmonics

Create a triangle signal and plot a 3 period segment.

In [ ]:
signal = thinkdsp.TriangleSignal(200)
duration = signal.period*3
segment = signal.make_wave(duration, framerate=10000)
segment.plot()

Make a wave and play it.

In [ ]:
wave = signal.make_wave(duration=0.5, framerate=10000)
wave.apodize()
wave.make_audio()

Compute its spectrum and plot it.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()

Make a square signal and plot a 3 period segment.

In [ ]:
signal = thinkdsp.SquareSignal(200)
duration = signal.period*3
segment = signal.make_wave(duration, framerate=10000)
segment.plot()
thinkplot.config(ylim=[-1.05, 1.05], legend=False)

Make a wave and play it.

In [ ]:
wave = signal.make_wave(duration=0.5, framerate=10000)
wave.apodize()
wave.make_audio()

Compute its spectrum and plot it.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()

Create a sawtooth signal and plot a 3 period segment.

In [ ]:
signal = thinkdsp.SawtoothSignal(200)
duration = signal.period*3
segment = signal.make_wave(duration, framerate=10000)
segment.plot()

Make a wave and play it.

In [ ]:
wave = signal.make_wave(duration=0.5, framerate=10000)
wave.apodize()
wave.make_audio()

Compute its spectrum and plot it.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()

### Aliasing

Make a cosine signal at 4500 Hz, make a wave at framerate 10 kHz, and plot 5 periods.

In [ ]:
framerate = 10000

signal = thinkdsp.CosSignal(4500)
duration = signal.period*5
segment = signal.make_wave(duration, framerate=framerate)
segment.plot()

Make a cosine signal at 5500 Hz, make a wave at framerate 10 kHz, and plot the same duration.

With framerate 10 kHz, the folding frequency is 5 kHz, so a 4500 Hz signal and a 5500 Hz signal look exactly the same.

In [ ]:
signal = thinkdsp.CosSignal(5500)
segment = signal.make_wave(duration, framerate=framerate)
segment.plot()

Make a triangle signal and plot the spectrum.  See how the harmonics get folded.

In [ ]:
signal = thinkdsp.TriangleSignal(1100)
segment = signal.make_wave(duration=0.5, framerate=10000)
spectrum = segment.make_spectrum()
spectrum.plot()

## Amplitude and phase

Make a sawtooth wave.

In [ ]:
signal = thinkdsp.SawtoothSignal(500)
wave = signal.make_wave(duration=1, framerate=10000)
segment = wave.segment(duration=0.005)
segment.plot()

Play it.

In [ ]:
wave.make_audio()

Extract the wave array and compute the real FFT (which is just an FFT optimized for real inputs).

In [ ]:
import numpy as np

hs = np.fft.rfft(wave.ys)
hs

Compute the frequencies that match up with the elements of the FFT.

In [ ]:
n = len(wave.ys)                 # number of samples
d = 1 / wave.framerate           # time between samples
fs = np.fft.rfftfreq(n, d)
fs

Plot the magnitudes vs the frequencies.

In [ ]:
magnitude = np.absolute(hs)
thinkplot.plot(fs, magnitude)

Plot the phases vs the frequencies.

In [ ]:
angle = np.angle(hs)
thinkplot.plot(fs, angle)

## What does phase sound like?

Shuffle the phases.

In [ ]:
import random
random.shuffle(angle)
thinkplot.plot(fs, angle)

Put the shuffled phases back into the spectrum.  Each element in `hs` is a complex number with magitude $A$ and phase $\phi$, with which we can compute $A e^{i \phi}$

In [ ]:
i = complex(0, 1)
spectrum = wave.make_spectrum()
spectrum.hs = magnitude * np.exp(i * angle)

Convert the spectrum back to a wave (which uses irfft).

In [ ]:
wave2 = spectrum.make_wave()
wave2.normalize()
segment = wave2.segment(duration=0.005)
segment.plot()

Play the wave with the shuffled phases.

In [ ]:
wave2.make_audio()

For comparison, here's the original wave again.

In [ ]:
wave.make_audio()

Although the two signals have different waveforms, they have the same frequency components with the same amplitudes.  They differ only in phase.

## Aliasing interaction

The following interaction explores the effect of aliasing on the harmonics of a sawtooth signal.

In [ ]:
def view_harmonics(freq, framerate):
    signal = thinkdsp.SawtoothSignal(freq)
    wave = signal.make_wave(duration=0.5, framerate=framerate)
    spectrum = wave.make_spectrum()
    spectrum.plot(color='blue')
    thinkplot.show(xlabel='frequency', ylabel='amplitude')
    
    display(wave.make_audio())

In [ ]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

slider1 = widgets.FloatSlider(min=100, max=10000, value=100, step=100)
slider2 = widgets.FloatSlider(min=5000, max=40000, value=10000, step=1000)
interact(view_harmonics, freq=slider1, framerate=slider2);

# Exercises

## 2.2

 A sawtooth signal has a waveform that ramps up linearly from -1 to 1, then drops to -1 and repeats. See http://en.wikipedia.org/wiki/Sawtooth_wave

 Write a class called `SawtoothSignal` that extends `Signal` and provides `evaluate` to evaluate a sawtooth signal.

 Compute the spectrum of a sawtooth wave. How does the harmonic structure compare to triangle and square waves? 

From solutions:

In [ ]:
import math
PI2 = 2 * math.pi

In [ ]:
class SawtoothSignal(thinkdsp.Sinusoid):
    """Represents a sawtooth signal."""
    
    def evaluate(self, ts):
        """Evaluates the signal at the given times.

        ts: float array of times
        
        returns: float wave array
        """
        cycles = self.freq * ts + self.offset / PI2
        frac, _ = np.modf(cycles)
        ys = thinkdsp.normalize(thinkdsp.unbias(frac), self.amp)
        return ys

In [ ]:
sawtooth = SawtoothSignal().make_wave(duration=0.5, framerate=40000)
sawtooth.make_audio()

In [ ]:
sawtooth.make_spectrum().plot()

Compared to a square wave, the sawtooth drops off similarly, but it includes both even and odd harmonics.  Notice that I had to cut the amplitude of the square wave to make them comparable.

In [ ]:
sawtooth.make_spectrum().plot(color='gray')
square = thinkdsp.SquareSignal(amp=0.5).make_wave(duration=0.5, framerate=40000)
square.make_spectrum().plot()

Compared to a triangle wave, the sawtooth doesn't drop off as fast.

In [ ]:
sawtooth.make_spectrum().plot(color='gray')
triangle = thinkdsp.TriangleSignal(amp=0.79).make_wave(duration=0.5, framerate=40000)
triangle.make_spectrum().plot()

Specifically, the harmonics of the triangle wave drop off in proportion to $1/f^2$, while the sawtooth drops off like $1/f$.

## 2.3

Make a square signal at 1100 Hz and make a wave that samples
it at 10000 frames per second. If you plot the spectrum, you can see that
most of the harmonics are aliased. When you listen to the wave, can you
hear the aliased harmonics?

In [ ]:
# 1100 HZ square signal sampled at 44100 Hz
square = thinkdsp.SquareSignal(1100).make_wave(duration=0.5, framerate=44000)
square.make_spectrum().plot()

In [ ]:
square.make_spectrum().peaks()[:10]

In [ ]:
# 1100 HZ square signal sampled at 11kHz
square = thinkdsp.SquareSignal(1100).make_wave(duration=5, framerate=10000)
square.make_spectrum().plot()

In [ ]:
square.make_spectrum().peaks()[:10]

Make it bun dem:

In [ ]:
square.make_audio()

## 2.4

If you have a spectrum object, spectrum, and print the first few
values of spectrum.fs, you’ll see that they start at zero. So spectrum.hs[0]
is the magnitude of the component with frequency 0. But what does that
mean?

Try this experiment:

1. Make a triangle signal with frequency 440 and make a Wave with duration 0.01 seconds. Plot the waveform.

2. Make a Spectrum object and print spectrum.hs[0]. What is the amplitude and phase of this component?

3. Set spectrum.hs[0] = 100. Make a Wave from the modified Spectrum and plot it. What effect does this operation have on the waveform?

1. Make a triangle signal with frequency 440 and make a Wave with duration 0.01 seconds. Plot the waveform.

In [ ]:
triangle = thinkdsp.TriangleSignal().make_wave(duration=0.01)
triangle.plot()

2. Make a Spectrum object and print spectrum.hs[0]. What is the amplitude and phase of this component?

In [ ]:
spectrum = triangle.make_spectrum()
spectrum.hs[0]
# The first element of the spectrum is a complex number close to zero.

In [ ]:
spectrum.hs[0] = 100
triangle.plot(color='gray')
spectrum.make_wave().plot()
# If we add to the zero-frequency component, 
#it has the effect of adding a vertical offset to the wave.

3. Set spectrum.hs[0] = 100. Make a Wave from the modified Spectrum and plot it. What effect does this operation have on the waveform?

The zero-frequency component is the total of all the values in the signal, as we'll see when we get into the details of the DFT.  If the signal is unbiased, the zero-frequency component is 0.  In the context of electrical signals, the zero-frequency term is called the DC offset; that is, a direct current offset added to an AC signal.

## 2.5

Write a function that takes a Spectrum as a parameter and
modifies it by dividing each element of hs by the corresponding frequency
from fs. Hint: since division by zero is undefined, you might want to set
spectrum.hs[0] = 0.

Test your function using a square, triangle, or sawtooth wave.

1. Compute the Spectrum and plot it.

2. Modify the Spectrum using your function and plot it again.

3. Make a Wave from the modified Spectrum and listen to it. What effect
does this operation have on the signal?


In [ ]:
def filter_spectrum(spectrum):
    spectrum.hs /= spectrum.fs
    spectrum.hs[0] = 0

In [ ]:
# Traingle wave
wave = thinkdsp.TriangleSignal(freq=440).make_wave(duration=0.5)
wave.make_audio()

In [ ]:
high = 10000
spectrum = wave.make_spectrum()
spectrum.plot(high=high, color='gray')
filter_spectrum(spectrum)
spectrum.scale(440)
spectrum.plot(high=high)

In [ ]:
filtered = spectrum.make_wave()
filtered.make_audio()

The filter clobbers the harmonics, so it acts like a low pass filter.
The triangle wave now sounds almost like a sine wave.

In [ ]:
spectrum.plot(high=high, color='gray')
filter_spectrum(spectrum)
filter_spectrum(spectrum)
spectrum.scale(440)
spectrum.plot(high=high)

## 2.6

 Triangle and square waves have odd harmonics only; the sawtooth wave has both even and odd harmonics. The harmonics of the square
and sawtooth waves drop off in proportion to $1/f$ ; the harmonics of the triangle wave drop off like $1/f^2$. Can you find a waveform that has even and
odd harmonics that drop off like $1/f^2$

Hint: There are two ways you could approach this: you could construct the
signal you want by adding up sinusoids, or you could start with a signal
that is similar to what you want and modify it.

In [ ]:
# One option is to start with a sawtooth wave, which has all of the harmonics we need:
freq = 500
signal = thinkdsp.SawtoothSignal(freq=freq)
wave = signal.make_wave(duration=0.5, framerate=20000)
wave.make_audio()


In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()

If we apply the filter we wrote in the previous exercise, we can make the harmonics drop off like $1/f^2$.

In [ ]:
spectrum.plot(color='gray')
filter_spectrum(spectrum)
spectrum.scale(freq)
spectrum.plot()

In [ ]:
wave = spectrum.make_wave()
wave.make_audio()